In [1]:
from GCN_model import GCN

import torch
import torch_geometric

import torch.nn as nn
import torch_geometric.datasets as datasets

In [2]:
dataset = datasets.Planetoid(
    root="./",
    name='Cora',
    split="public",
    transform=torch_geometric.transforms.GCNNorm()
  )
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


c:\Users\ckaz3\miniconda3\envs\grl_mini_project\Lib\site-packages\torch_geometric\data\in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [3]:
def evaluate(
        model,
        data,
        mask):
    
    edge_index = torch_geometric.EdgeIndex(dataset.edge_index)
    adj_matrix = edge_index.to_dense()
    
    output = model(data.x, adj_matrix)

    output = torch.argmax(output[mask], dim=1)
    target = data.y[mask]

    return torch.mean((output == target).float())

In [4]:
def train(
    num_layers: int,
    aug_adj_type: str,
    lr: int,
    weight_decay: int,
    num_epochs: int,
    dataset,
    verbose: bool = True
) -> torch.nn.Module:
    """
    This function trains a node classification model and returns the trained model object.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"

    data = dataset.data
    data = data.to(device)

    # calculate adjacency matrix
    edge_index = torch_geometric.EdgeIndex(dataset.edge_index)
    adj_matrix = edge_index.to_dense()

    model = GCN(
        in_channels=dataset.num_features,
        hidden_channels=64,
        out_channels=dataset.num_classes,
        num_layers=num_layers,
        aug_adj_type=aug_adj_type,
        dropout=0.3,
    ).to(device)

    losses = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    early_stop_counter = 0
    prev_val_acc = 1.1

    for i in range(num_epochs):
        optimizer.zero_grad()

        outputs = model(data.x, adj_matrix)

        loss = criterion(outputs[data.train_mask], data.y[data.train_mask])
        losses.append(loss)

        loss.backward()
        optimizer.step()

        # check for early-stopping
        val_acc = evaluate(model, data, data.val_mask)

        if val_acc < prev_val_acc:
            if prev_val_acc != 1.1:
                early_stop_counter += 1
            prev_val_acc = val_acc

            if early_stop_counter == 5:
                break
        else:
            prev_val_acc = 1.1
            early_stop_counter = 0

        if i % 5 == 0 and verbose:
            print(f"Epoch: {i}, Loss: {loss.item():.4f}, Val Acc: {val_acc:.4f}")

    if verbose:
        print(f"Epoch: {i}, Loss: {loss.item():.4f}, Val Acc: {val_acc:.4f}")
    return model

In [5]:
from tqdm import tqdm
import numpy as np
import pickle

In [7]:
layers = [0, 1] 
# layers.extend(np.arange(2, 12, 2))

study_data = {"symmetric": {}, "degree": {}, "adjacency": {}, "random walk": {}}

for aug_adj_type in tqdm(study_data):
    for num_layers in tqdm(layers):
        # train model
        torch.manual_seed(2025) 
        model = train(
            num_layers=num_layers,
            aug_adj_type=aug_adj_type,
            lr=0.001,
            weight_decay=0.0001,
            num_epochs=100,
            dataset=dataset,
            verbose=False
        )

        device = "cuda" if torch.cuda.is_available() else "cpu"

        data = dataset.data
        data = data.to(device)

        final_val_acc = evaluate(model, data, data.val_mask)

        study_data[aug_adj_type][num_layers] = final_val_acc
        
        f = open("data.pkl", "wb")
        pickle.dump(study_data, f)
        f.close()

100%|██████████| 4/4 [03:04<00:00, 46.03s/it]
